# Part 1

## Import libraries

In [57]:
import requests
import lxml.html as lh
import pandas as pd

## Scrape table from wikipeida

In [58]:
my_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(my_url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content().rstrip('\n')
    col.append((name,[]))


In [59]:
for j in range(1,len(tr_elements)):

    T=tr_elements[j]
    

    if len(T)!=3:
        break
    

    i=0
    

    for t in T.iterchildren():
        data=t.text_content().rstrip('\n') 

        if i>0:

            try:
                data=int(data)
            except:
                pass

        col[i][1].append(data)

        i+=1

In [60]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [61]:
# there is one row at the bottom of this dataframe from another table. Remove it
df=df.drop([180])
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


## Remove NA from Borough

In [62]:
df=df[df.Borough!='Not assigned'].reset_index(drop=True)

In [63]:
df[df.Neighbourhood=='Not assigned']
# There is no 'Not assigned' neighbourhood

,Postal Code,Borough,Neighbourhood


In [64]:
df.shape

(103, 3)

# Part 2

In [65]:
"""
!conda install -c conda-forge geocoder
import geocoder # import geocoder
import numpy as np
df[['latitude','longitude']]=pd.DataFrame([[np.nan,np.nan]])

for i in range(0,len(df['Postal Code'])-1):
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    df['latitude'][i] = lat_lng_coords[0]
    df['longitude'][i] = lat_lng_coords[1]

I tried to install geocoder but failed
"""   


"\n!conda install -c conda-forge geocoder\nimport geocoder # import geocoder\nimport numpy as np\ndf[['latitude','longitude']]=pd.DataFrame([[np.nan,np.nan]])\n\nfor i in range(0,len(df['Postal Code'])-1):\n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n\n    df['latitude'][i] = lat_lng_coords[0]\n    df['longitude'][i] = lat_lng_coords[1]\n\nI tried to install geocoder but failed\n"

In [66]:
# get latitude and longitude data of postal code from this link https://cocl.us/Geospatial_data
!wget -q -O 'geospatial_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [67]:
geo_data = pd.read_csv('geospatial_data.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [68]:
df = pd.merge(df,geo_data,how='left',on='Postal Code')

In [69]:
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
